In [1]:
import altair as alt
import pandas as pd
from vega_datasets import data

pd.set_option("display.max_columns", None)
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Create Helper for Saving Outputs

In [30]:
def save_chart(chart, filename, fmt="svg", scale=2):
    filepath = f"charts/{filename}.{fmt}"
    chart.save(filepath, format=fmt, scale=scale)

## Load & Prep Data

In [3]:
arrests_df = pd.read_excel("data/arrests-latest.xlsx")

In [ ]:
# roll up duplicates
# Asked AI: how to keep the first value of duplicate rows in pandas and it gave this: df.drop_duplicates(subset=["A"], keep="first")
arrests_dedupe = arrests_df.drop_duplicates(subset=["unique_identifier"], keep="first")
# print(len(arrests_df)) # 291722
# print(len(arrests_dedupe)) # 274017

## Create Visualizations

In [ ]:
# Disable "..." button on top right of charts
alt.renderers.set_embed_options(actions=False)


# Configure global look (set palette)
# Asked AI: what decorator can i use to set global theme with altair:
# @alt.theme.register("my_global_theme") and gave a sample theme function that I adjusted for my teal theme
@alt.theme.register("teal_theme", enable=True)
def teal_theme():
    return {
        "config": {
            "range": {
                "category": {"scheme": "teals"},
                "heatmap": {"scheme": "teals"},
            },
            "mark": {"color": "#008080"},
        },
    }


In [45]:
# limit df to only needed cols to speed up run time
arrests_date = arrests_dedupe[["apprehension_date"]].copy()

chart = (
    alt.Chart(arrests_date)
    .transform_window(
        sort=[alt.SortField("apprehension_date", order="ascending")],
        frame=[None, 0],
        cumulative_count="count()",
    )
    .mark_line()
    .encode(
        alt.X("yearmonthdate(apprehension_date):T").title("Date"),
        alt.Y("cumulative_count:Q").title("Cumulative Arrests"),
    )
)

save_chart(chart, "cumulative_arrests")

In [46]:
base = (
    alt.Chart(arrests_date)
    .mark_line()
    .encode(
        alt.X("yearmonth(apprehension_date):T").title("Date"),
        alt.Y("count():Q").title("Total Arrests"),
    )
    .properties(width=400)
)

rule = (
    alt.Chart(
        pd.DataFrame(
            {
                "apprehension_date": [pd.Timestamp("2025-01-20")],
                "label": ["Trump Inaugurated"],
            }
        )
    )
    .mark_rule(color="red", strokeWidth=2)
    .encode(x="yearmonth(apprehension_date):T", size=alt.value(2))
)

text = (
    alt.Chart(
        pd.DataFrame(
            {
                "apprehension_date": [pd.Timestamp("2025-01-20")],
                "y": [arrests_date["apprehension_date"].value_counts().max()],
                "label": ["Trump Inaugurated"],
            }
        )
    )
    .mark_text(align="left", dx=5, dy=-5, color="red")
    .encode(x="yearmonth(apprehension_date):T", y=alt.value(0), text="label")
)

chart = (base + rule + text).properties(title="ICE Arrests (Sept 2023 - July 2025)")
save_chart(chart, "arrests_over_time")

In [33]:
# edit using this: https://altair-viz.github.io/gallery/groupby-map.htmls
## look into better source for state centroids than using capitals
# see if you can do a version of this that normalizes for population (need secondary dataset)

arrest_states = arrests_dedupe.copy()

# Clean state names and aggregate to counts
# Drop missing / unknowns
arrest_states = arrest_states.dropna(subset=["apprehension_state"])

# Normalize to Title Case to match vega_datasets' "state" names
# (eg 'SOUTH CAROLINA' -> 'South Carolina')
arrest_states["state"] = (
    arrest_states["apprehension_state"].astype(str).str.strip().str.title()
)

# Fix small title-case quirks (e.g. 'Of' in DC)
arrest_states["state"] = arrest_states["state"].replace(
    {
        "District Of Columbia": "District of Columbia",
    }
)

# drop missing values
df = arrest_states.dropna(subset=["state"])

# Count arrests by state
by_state = (
    df.groupby("state", as_index=False).size().rename(columns={"size": "arrests"})
)

# Get map & centroids from vega_datasets
# Base US states topology
states = alt.topo_feature(data.us_10m.url, "states")

# State "centroids": use the state capital coordinates (proxy)
capitals = pd.DataFrame(data.us_state_capitals())  # columns: state, city, lat, lon

centroids = capitals[["state", "lat", "lon"]].copy()

# Join your counts to centroids
plot_df = pd.merge(centroids, by_state, on="state", how="inner")

# Background map
background = (
    alt.Chart(states)
    .mark_geoshape(fill="#eeeeee", stroke="#ffffffb0")
    .project("albersUsa")
)

# Proportional symbols
points = (
    alt.Chart(plot_df)
    .mark_circle(opacity=0.7)
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
        size=alt.Size("arrests:Q", title="Arrests", scale=alt.Scale(range=[0, 2500])),
    )
    .project("albersUsa")
)


chart = (background + points).properties(
    title="ICE Arrests by State (Sept 2023 - July 2025)", width=500, height=400
)

save_chart(chart, "arrests_by_state")


### Demographics

In [ ]:
# departure country: Might scrap -- not very interesting
arrests_departure = arrests_dedupe[["departure_country"]].copy()
top5 = (
    arrests_departure["departure_country"]
    .value_counts()
    .nlargest(5)
    .rename_axis("departure_country")
    .reset_index(name="arrests")
)

# create chart
chart = (
    alt.Chart(top5)
    .mark_bar()
    .encode(
        alt.X("departure_country:N", sort="-y").title("Departure Country"),
        alt.Y("arrests:Q").title("Arrests"),
    )
)

save_chart(chart, "departure_country")


In [35]:
# prep df
arrests_citizen = arrests_dedupe[["citizenship_country"]].copy()
top5 = (
    arrests_citizen["citizenship_country"]
    .value_counts()
    .nlargest(5)
    .rename_axis("citizenship_country")
    .reset_index(name="arrests")
)

# create chart
chart = (
    alt.Chart(top5)
    .mark_bar()
    .encode(
        alt.X("citizenship_country:N", sort="-y").title("Citizenship Country"),
        alt.Y("arrests:Q").title("Arrests"),
    )
)

save_chart(chart, "citizenship_country")

In [ ]:
# top5 citizenship country arrests make up X% of total arrests

# prep data
total = arrests_dedupe.shape[0]
top5_sum = top5["arrests"].sum()
top5_perc = (top5_sum / total) * 100
remaining_perc = ((total - top5_sum) / total) * 100

# Asked AI: whats the syntax for creating pandas df and it gave the following:
# data = {
#     "name": ["Alice", "Bob", "Charlie"],
#     "age": [25, 30, 35],
#     "city": ["NYC", "LA", "Chicago"]
# }

# df = pd.DataFrame(data)

pie_data = pd.DataFrame(
    {
        "category": ["Top 5 Countries", "All Other Countries"],
        "arrest_perc": [top5_perc, remaining_perc],
    }
)

# create chart
chart = (
    alt.Chart(pie_data)
    .mark_arc()
    .encode(
        alt.Theta("arrest_perc:Q", title="Percent of Total Arrests"),
        alt.Color("category:N", title="Citizenship Country"),
    )
    .properties(
        title="Share of Total Arrests by Citizenship Country: Top 5 vs. All Other"
    )
)

save_chart(chart, "pie_top5_countries")

In [49]:
methods = arrests_dedupe[["apprehension_method", "apprehension_date"]].copy()


# bucket apprehension methods
bucket_map = {
    "CAP Local Incarceration": "CAP",
    "CAP Federal Incarceration": "CAP",
    "CAP State Incarceration": "CAP",
    "Non-Custodial Arrest": "ERO",
    "Custodial Arrest": "ERO",
    "ERO Reprocessed Arrest": "ERO",
    "Other efforts": "Task Force / Other Agency",
    "Other Task Force": "Task Force / Other Agency",
    "Other Agency (turned over to INS)": "Task Force / Other Agency",
    "Law Enforcement Agency Response Unit": "Task Force / Other Agency",
    "Organized Crime Drug Enforcement Task Force": "Task Force / Other Agency",
    "287(g) Program": "287g / Probation",
    "Probation and Parole": "287g / Probation",
    "Patrol Border": "Border / Patrol",
    "Patrol Interior": "Border / Patrol",
    "Located": "Border / Patrol",
    "Worksite Enforcement": "Worksite / Inspections",
    "Inspections": "Worksite / Inspections",
    "Anti-Smuggling": "Worksite / Inspections",
    "Traffic Check": "Worksite / Inspections",
    "Transportation Check Aircraft": "Transportation",
    "Transportation Check Bus": "Transportation",
    "Transportation Check Passenger Train": "Transportation",
    "Presented During Inspection": "Transportation",
    "Boat Patrol": "Transportation",
    "Crewman/Stowaway": "Transportation",
    "Criminal Alien Program": "Miscellaneous",
}

# asked AI: how to extract year from date column in pandas: df["year"] = df["date"].dt.year
methods["year"] = methods["apprehension_date"].dt.year
methods["method_bucket"] = (
    methods["apprehension_method"].map(bucket_map).fillna("Miscellaneous")
)

# group by method and arrest year
rolled = methods.groupby(["method_bucket", "year"]).size().reset_index(name="count")

chart = (
    alt.Chart(rolled)
    .mark_rect()
    .encode(
        alt.X("year:N", title="Arrest Year"),
        alt.Y("method_bucket:N", title="Apprehension Method"),
        alt.Color("count:Q", title="Arrest Count"),
    )
    .properties(width=200)
)

save_chart(chart, "heatmap_method")

In [38]:
# Age Distribution (histogram)
age_df = arrests_dedupe.copy()
# create age col
age_df["age"] = 2025 - age_df["birth_year"]
age_df = age_df[["age"]]

base = alt.Chart(age_df)

bar = base.mark_bar().encode(
    alt.X("age:Q", bin=True, axis=alt.Axis(title="Age (years)")),
    alt.Y("count()", axis=alt.Axis(title="Count of Arrests")),
)

# Mean line (red)
rule = base.mark_rule(color="red", strokeWidth=5).encode(x="mean(age):Q")

# Label for mean line
label = base.mark_text(
    align="left",  # position to the right of the line
    dx=5,  # horizontal offset
    dy=-5,  # vertical offset (move up)
    color="red",
).encode(
    x="mean(age):Q",
    y=alt.value(0),  # y=0 or small positive value keeps it near x-axis
    text=alt.value("Mean Age"),
)

chart = (bar + rule + label).properties(title="Distribution of Arrests by Age")

save_chart(chart, "age_histogram")


In [39]:
# time from arrest to deportation (subset of data: only deported persons)
# print(arrests_dedupe["final_order_date"].isnull().sum()) # 107250
# print(len(arrests_dedupe)) # 274017
# ~1/3 final_order_date values are missing, exclude from analysis and make a note on chart

final_date_df = (arrests_dedupe.copy()).dropna(
    subset=["departed_date", "apprehension_date"]
)

final_date_df["days_to_departure"] = (
    final_date_df["departed_date"] - final_date_df["apprehension_date"]
).dt.days.astype("Int64")
len(final_date_df[final_date_df["days_to_departure"] > 0])
# final_date_df[final_date_df["days_to_departure"] < 0]

# final_date_df["decision_days"].mean()


147268

In [40]:
yes_count = (final_date_df["final_order_yes_no"] == "YES").sum()
no_count = (final_date_df["final_order_yes_no"] == "NO").sum()
total = (final_date_df["final_order_yes_no"]).count()

pie_deportations = pd.DataFrame(
    {
        "deportation_status": ["Yes", "No"],
        "perc": [(yes_count / total), (no_count / total)],
    }
)

chart = (
    alt.Chart(pie_deportations)
    .mark_arc()
    .encode(
        alt.Theta("perc:Q", title="Percent of Total Arrests"),
        alt.Color("deportation_status:N", title="Deportation Status"),
    )
    .properties(title="Share of Deportations")
)  # Or "Share of Arrests That Led To Deportation"

save_chart(chart, "pie_deportations")

In [41]:
# Density: time taken between arrest and deportation
# limit to only deported arrests & "days_to_departure" > 0
deported_df = final_date_df[
    (final_date_df["final_order_yes_no"] == "YES")
    & (final_date_df["days_to_departure"] > 0)
]

mean_days = deported_df["days_to_departure"].mean()

# create chart
base = (
    alt.Chart(deported_df)
    .transform_density(
        "days_to_departure",
        as_=["days_to_departure", "density"],
    )
    .mark_area()
    .encode(
        alt.X("days_to_departure:Q", title="Number of Days"),
        alt.Y("density:Q", title="Density"),
    )
)

mean_line = (
    alt.Chart(pd.DataFrame({"days_to_departure": [mean_days]}))
    .mark_rule(color="red", strokeWidth=2)
    .encode(x="days_to_departure:Q")
)

mean_label = (
    alt.Chart(
        pd.DataFrame(
            {
                "days_to_departure": [mean_days],
                "label": [f"Mean = {mean_days:.0f} days"],
            }
        )
    )
    .mark_text(
        align="left",
        angle=0,
        dx=5,  # move right of the line
        dy=5,  # move slightly up
        color="red",
    )
    .encode(
        x="days_to_departure:Q",
        y=alt.value(0.018),  # roughly top of the chart
        text="label",
    )
)

chart = (base + mean_line + mean_label).properties(
    title="Density of Days Between Arrest and Departure", width=400
)

save_chart(chart, "density_days")